In [1]:
import os
import sys
import numpy as np
import pandas as pd
import random

from argparse import ArgumentParser
import datetime

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.backends.cudnn as cudnn

from pytvision.transforms import transforms as mtrans
from pytvision import visualization as view

sys.path.append('../')
from torchlib.datasets.datasets import ATLASDataset 
from torchlib.neuralnet import NeuralNetClassifier
from misc import (get_transforms_det, get_transforms_hflip, get_transforms_vflip )

from sklearn import metrics

In [2]:
pathname = os.path.expanduser( '~/.datasets' )
projectnet='../out/netruns'
no_cuda=False
seed=1
gpu=0
batch_size=60
workers=10
num_input_channels=3
parallel=False

print(pathname)
print(projectnet)

/home/fernandez/.datasets
../out/netruns


In [3]:
!ls ../out/netruns/

atlas_baseline_inception_v4_mix_adam_atlas_004
atlas_baseline_preactresnet18_bcewl_adam_atlas_001
atlas_baseline_preactresnet18_mix_sgd_atlas_002
atlas_baseline_preactresnet18_mix_sgd_atlas_003
atlas_baseline_resnet101_mix_adam_atlas_006
atlas_baseline_resnet152_mix_adam_atlas_006
atlas_baseline_resnet18_mix_adam_atlas_005
atlas_baseline_resnet50_mix_adam_atlas_006
results
test_atlas_baseline_inception_v4_mix_adam_atlas_004_atlas_0_dp.csv
test_atlas_baseline_inception_v4_mix_adam_atlas_004_atlas_1_dp.csv
test_atlas_baseline_inception_v4_mix_adam_atlas_004_atlas_2_dp.csv


In [4]:
name = 'atlas_baseline_resnet50_mix_adam_atlas_006'
project = os.path.join(projectnet, name)
pathnameout = os.path.join(projectnet, 'results' )
model = 'model_best.pth.tar' #'chk000025.pth.tar', model_best
pathmodels = os.path.join(project,'models',model)

print(project)
print(pathmodels)

../out/netruns/atlas_baseline_resnet50_mix_adam_atlas_006
../out/netruns/atlas_baseline_resnet50_mix_adam_atlas_006/models/model_best.pth.tar


In [5]:
# neuralnet
network = NeuralNetClassifier(
    patchproject=project,
    nameproject=name,
    no_cuda=no_cuda,
    seed=seed,
    gpu=gpu,
    parallel=parallel,
    )

cudnn.benchmark = True

# load model
if network.load( pathmodels ) is not True:
    assert(False)

Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


=> loading checkpoint '../out/netruns/atlas_baseline_resnet50_mix_adam_atlas_006/models/model_best.pth.tar'
=> loaded checkpoint for resnet50 arch!


In [6]:
pathdata='~/.kaggle/competitions/human-protein-atlas-image-classification'
name_dataset='atlas'
subset='test'

tta_preprocess = [ 
    get_transforms_det( network.size_input ), 
    get_transforms_hflip(network.size_input), 
    get_transforms_vflip(network.size_input), 
    #get_transforms_gray(network.size_input),
    #get_transforms_aug(network.size_input),
    #get_transforms_aug2(network.size_input)
    ]

dataloaders = []
for transform in tta_preprocess:    
    # test dataset    
    data = ATLASDataset(        
        path=pathdata, 
        train=False,
        folders_images='test', 
        metadata='sample_submission.csv',
        #count=100,
        num_channels=3,
        transform=transform,
        )
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=workers )   
    dataloaders.append(dataloader)

print(len(dataloaders))

3


In [8]:
random.seed( seed )
files = [ f for f in sorted(os.listdir(pathnameout)) if f.split('.')[-1] == 'csv' ]

for i,data in enumerate(dataloaders):
    Id, Yhat, Y = network.predict( data )
    df = pd.DataFrame( np.concatenate((Id[:,np.newaxis], Yhat), axis=1) )
    df.to_csv( os.path.join(pathnameout , '{}_{}_{}_{}_dp.csv'.format(subset,name,name_dataset, i + len(files))), index=False, encoding='utf-8')            

print('dir: {}'.format(pathnameout))
print('DONE!!!')

100%|██████████| 196/196 [01:28<00:00,  2.21it/s]


dir: ../out/netruns/results
DONE!!!


In [9]:
files = [ f for f in sorted(os.listdir(pathnameout)) if f.split('.')[-1] == 'csv' ]; 
print(files)

l = len(files)
dp =[]; ids=[]
for f in files:  
    mdata = pd.read_csv( os.path.join(pathnameout , f )  )
    dpdata = mdata.as_matrix()
    ids.append(dpdata[:,0])    
    dp.append( dpdata[:,1:] )
    
dp = np.array(dp).transpose((1,2,0))
ids = np.array(ids)

print(dp.shape)

['test_atlas_baseline_resnet18_mix_adam_atlas_005_atlas_0_dp.csv', 'test_atlas_baseline_resnet18_mix_adam_atlas_005_atlas_1_dp.csv', 'test_atlas_baseline_resnet18_mix_adam_atlas_005_atlas_2_dp.csv', 'test_atlas_baseline_resnet50_mix_adam_atlas_006_atlas_3_dp.csv', 'test_atlas_baseline_resnet50_mix_adam_atlas_006_atlas_4_dp.csv', 'test_atlas_baseline_resnet50_mix_adam_atlas_006_atlas_5_dp.csv']
(11702, 28, 6)


In [10]:
def mean_ruler( dp ):
    """
    Ecuation. Josef Kittler [18]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Median Ruler
    Args:
        @dp: []_nxcxl
    """
    p = dp.mean(2)
    return p

def product_ruler( dp, P=None ):
    """
    Ecuation. Josef Kittler [7]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Product Rule    
    P^{(-(R-1))}( w_j )\prod_iP(w_j|x_i)  = \max_k P^{(-(R-1))}( w_k ) \prod_i
    P(w_k|x_i) (1)
    which under the assumption of equal priors, simplifies to the following:
    \prod_iP(w_j|x_i)  = \max_k \prod_i P(w_k|x_i) (2)    
    Args:
        @dp: []_nxcxl
        @P: class prior []_c
    """    
    p = dp.prod(2)
    if P is not None:
        l = dp.shape[2]
        p = P**-(l-1)*p
    return p

    

In [11]:
iDs = ids[0,:]
P = mean_ruler(dp)
# P = product_ruler(dp)
th=0.1

#pred = np.argmax(p, axis=1)
pred = [ ' '.join( map(str,np.where( p > th )[0]) ) for p in P]

print(dp.shape)
print(P.shape)
#print(pred)

(11702, 28, 6)
(11702, 28)


In [12]:
ths = np.array([0.2375, 0.0375, 0.0875, 0.0375, 0.0125, 0.0375, 0.025, 0.05625,
       0.0, 0.0, 0.0, 0.025, 0.025, 0.0125, 0.025, 0.0, 0.01875, 0.00625,
       0.025, 0.00625, 0.00625, 0.08125, 0.0125, 0.06875, 0.00625, 0.15,
       0.0, 0.0])

In [16]:
submission_filepath = 'submission.csv'
th=ths + 0.3
submission = pd.read_csv('~/.kaggle/competitions/human-protein-atlas-image-classification/sample_submission.csv')

#submission = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0])  if len(np.where( yhat > th )[0])>0 else '0'  ) }  for (Id, yhat) in zip( iDs,Yhat) ]
submission_local = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( p > th )[0]) ) }  for (Id, p) in zip( iDs,P) ]
submission_local = pd.DataFrame( submission_local )
submission.loc[ submission['Id'] == submission_local['Id']   , 'Predicted'] = submission_local['Predicted']
submission.to_csv(submission_filepath, index=False, encoding='utf-8')
submission

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,25
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,4
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0 25
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 25
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0 25


In [17]:
!kaggle competitions submit -c human-protein-atlas-image-classification -f submission.csv -m "tta-resnet50+mix+512x512+extern+ep50+ep03+multth03"

100%|█████████████████████████████████████████| 469k/469k [00:01<00:00, 346kB/s]
Successfully submitted to Human Protein Atlas Image Classification